## Run test

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
#default_exp run.test

In [3]:
# %load ../../../HPA-competition-solutions/bestfitting/src/run/test.py

In [4]:
import os, sys
import argparse
from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
import torch.optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SequentialSampler
from torch.nn import DataParallel
import torch.nn.functional as F
from torch.autograd import Variable

from kgl_humanprotein.config.config import *
from kgl_humanprotein.utils.common_util import *
from kgl_humanprotein.networks.imageclsnet import init_network
from kgl_humanprotein.datasets.protein_dataset import ProteinDataset
from kgl_humanprotein.utils.augment_util import *
from kgl_humanprotein.utils.log_util import Logger

run on collie.local


In [5]:
#export
datasets_names = ['test', 'val']
split_names = ['random_ext_folds5', 'random_ext_noleak_clean_folds5']
augment_list = ['default', 'flipud', 'fliplr','transpose', 'flipud_lr',
                'flipud_transpose', 'fliplr_transpose', 'flipud_lr_transpose']

In [14]:
#export

def test(outdir, gpu_id='0', arch='class_densenet121_dropout', 
         num_classes=28, in_channels=4, img_size=768, crop_size=512, 
         batch_size=32, workers=3, fold=0, augment='default', seed=100,  
         seeds=None, dataset='test', split_name='random_ext_folds5',
         predict_epoch=None):  
    '''
    PyTorch Protein Classification
    
    Args:
        outdir (str): Destination where predicted result should be saved.
        gpu_id (str): GPU id used for predicting. Default: ``'0'``
        arch (str): Model architecture. Default: ``'class_densenet121_dropout)'``
        num_classes (int): Number of classes. Default: 28
        in_channels (int): In channels. Default: 4
        img_size (int):  Image size. Default: 768
        crop_size (int): Crop size. Default: 512
        batch_size (int): Train mini-batch size. Default: 32
        workers (int): Number of data loading workers. Default: 3
        fold (int): Index of fold. Default: 0
        augment (str):  Comma-separated string of one or more of 
            the following: ``'default'``, ``'flipud'``, ``'fliplr'``,
            ``'transpose'``, ``'flipud_lr'``, ``'flipud_transpose'``, 
            ``'fliplr_transpose'``, ``'flipud_lr_transpose'``.
            Default: ``'default'``
        seed (int):  Random seed. Default: 100
        seeds (str): Predict seed. Default: None
        dataset (str, optional): ``'test'``, or ``'val'``. Default: ``'test'``
        split_name (str, optional): ``'random_ext_folds5'``, or
            ``'random_ext_noleak_clean_folds5'``. Default: 'random_ext_folds5'
        predict_epoch (int): Number epoch to predict. Default: None
    '''
    if dataset not in datasets_names:
        print(f'`dataset` needs to be one of {datasets_names}.')
        raise

    if split_name not in split_names:
        print(f'`split_name` must be one of {split_names}.')
        raise

    log_out_dir = opj(RESULT_DIR, 'logs', out_dir, 'fold%d' % fold)
    if not ope(log_out_dir):
        os.makedirs(log_out_dir)
    log = Logger()
    log.open(opj(log_out_dir, 'log.submit.txt'), mode='a')

    predict_epoch = 'final' if predict_epoch is None else '%03d' % predict_epoch
    network_path = opj(RESULT_DIR, 'models', out_dir, 'fold%d' % fold, '%s.pth' % predict_epoch)

    submit_out_dir = opj(RESULT_DIR, 'submissions', out_dir, 'fold%d' % fold, 'epoch_%s' % predict_epoch)
    log.write(">> Creating directory if it does not exist:\n>> '{}'\n".format(submit_out_dir))
    if not ope(submit_out_dir):
        os.makedirs(submit_out_dir)

    # setting up the visible GPU
    if torch.cuda.is_available():
        os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    augment = augment.split(',')
    for augment_ in augment:
        if augment_ not in augment_list:
            raise ValueError('Unsupported or unknown test augmentation: {}!'.format(augment))

    model_params = {}
    model_params['architecture'] = arch
    model_params['num_classes'] = num_classes
    model_params['in_channels'] = in_channels
    model = init_network(model_params)

    log.write(">> Loading network:\n>>>> '{}'\n".format(network_path))
    checkpoint = torch.load(network_path)
    model.load_state_dict(checkpoint['state_dict'])
    log.write(">>>> loaded network:\n>>>> epoch {}\n".format(checkpoint['epoch']))

    # moving network to gpu and eval mode
    model = DataParallel(model)
    model.to(device)
    model.eval()

    # Data loading code
    if dataset == 'test':
        test_split_file = opj(DATA_DIR, 'split', 'train.csv')
    elif dataset == 'val':
        test_split_file = opj(DATA_DIR, 'split', split_name, 'random_valid_cv%d.csv' % fold)
    else:
        raise ValueError('Unsupported or unknown dataset: {}!'.format(dataset))
    test_dataset = ProteinDataset(
        test_split_file,
        img_size=img_size,
        is_trainset=(dataset != 'test'),
        return_label=False,
        in_channels=in_channels,
        transform=None,
        crop_size=crop_size,
        random_crop=False,
    )
    test_loader = DataLoader(
        test_dataset,
        sampler=SequentialSampler(test_dataset),
        batch_size=batch_size,
        drop_last=False,
        num_workers=workers,
        pin_memory=True,
    )
    return test_dataset

#     seeds = [seed] if seeds is None else [int(i) for i in seeds.split(',')]
#     for seed in seeds:
#         test_dataset.random_crop = (seed != 0)
#         for augment_ in augment:
#             test_loader.dataset.transform = eval('augment_%s' % augment_)
#             if crop_size > 0:
#                 sub_submit_out_dir = opj(submit_out_dir, '%s_seed%d' % (augment_, seed))
#             else:
#                 sub_submit_out_dir = opj(submit_out_dir, augment_)
#             if not ope(sub_submit_out_dir):
#                 os.makedirs(sub_submit_out_dir)
#             with torch.no_grad():
#                 predict(test_loader, model, sub_submit_out_dir, dataset)

def predict(test_loader, model, submit_out_dir, dataset):
    all_probs = []
    img_ids = np.array(test_loader.dataset.img_ids)
    for it, iter_data in tqdm(enumerate(test_loader, 0), total=len(test_loader)):
        images, indices = iter_data
        images = Variable(images.to(device), volatile=True)
        outputs = model(images)
        logits = outputs

        probs = F.sigmoid(logits).data
        all_probs += probs.cpu().numpy().tolist()
    img_ids = img_ids[:len(all_probs)]
    all_probs = np.array(all_probs).reshape(len(img_ids), -1)

    np.save(opj(submit_out_dir, 'prob_%s.npy' % dataset), all_probs)

    result_df = prob_to_result(all_probs, img_ids)
    result_df.to_csv(opj(submit_out_dir, 'results_%s.csv.gz' % dataset), index=False, compression='gzip')

def prob_to_result(probs, img_ids, th=0.5):
    probs = probs.copy()
    probs[np.arange(len(probs)), np.argmax(probs, axis=1)] = 1

    pred_list = []
    for line in probs:
        s = ' '.join(list([str(i) for i in np.nonzero(line > th)[0]]))
        pred_list.append(s)
    result_df = pd.DataFrame({ID: img_ids, PREDICTED: pred_list})
    return result_df

run on collie.local


In [15]:
# python test.py \
#             --out_dir external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds \
#             --gpu_id 0 --arch class_densenet121_dropout \
#             --img_size 768 --crop_size 512 --seeds 0,1,2,3 --batch_size 12 --fold 0 \
#             --augment

out_dir = 'external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds'

kwargs = dict(
    gpu_id = '0',
    arch = 'class_densenet121_dropout',
    img_size = 768,
    crop_size = 512,
    seeds = '0,1,2,3',
    batch_size = 12,
    fold = 0,
    augment = 'default,flipud,fliplr,transpose,flipud_lr,flipud_transpose,fliplr_transpose,flipud_lr_transpose')

In [23]:
ds = test(out_dir, **kwargs)

run on collie.local
>> Creating directory if it does not exist:
>> '../data/result/submissions/external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds/fold0/epoch_final'
>> Using pre-trained model.
>> Loading network:
>>>> '../data/result/models/external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds/fold0/final.pth'
>>>> loaded network:
>>>> epoch 52
../data/protein/test/images_768
../data/protein/train/external_v18_768


In [24]:
ds[5]

read_rgby


AssertionError: Cannot find ../data/protein/test/images_768/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0_5_red.png